# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df=pd.read_csv("output_data/cities.csv")
df=df.drop(["Unnamed: 0"], axis=1)
df

,city name,lat,temp,humidity,temp max,cloudiness,wind speed,lon
0,cabo san lucas,22.89,61.00,87,63.00,5,14.99,-109.91
1,busselton,-33.65,60.21,58,64.00,100,10.51,115.33
2,adrar,20.50,82.96,25,82.96,80,15.77,-10.07
3,ribeira grande,38.52,64.40,93,64.40,75,11.41,-28.70
4,avarua,-21.21,75.20,83,75.20,0,3.36,-159.78
...,...,...,...,...,...,...,...,...
95,deputatskiy,69.30,-14.71,81,-14.71,0,0.34,139.90
96,nalut,30.33,88.41,18,88.41,93,15.73,10.85
97,komsomolskiy,40.43,75.72,15,77.00,20,4.70,71.72
98,bakel,14.90,93.61,17,93.61,66,14.23,-12.46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

coordinates=df[["lat","lon"]]
# humidity_weight=df[["humidity"]]
df['humidity'] = df['humidity'].astype(float)
humidity_weight=df["humidity"]
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
vacay_df=df.loc[(df["temp max"]>70) & (df["temp max"]<80) & (df["cloudiness"]<30)]
clean_vacay_df=vacay_df.dropna()
clean_vacay_df.count()
clean_df=clean_vacay_df.reset_index()
clean_df


,index,city name,lat,temp,humidity,temp max,cloudiness,wind speed,lon
0,4,avarua,-21.21,75.20,83.0,75.20,0,3.36,-159.78
1,21,ahuimanu,21.44,73.18,54.0,78.80,20,6.93,-157.84
2,24,rikitea,-23.12,76.87,69.0,76.87,2,5.64,-134.97
3,33,yulara,-25.24,78.80,29.0,78.80,7,6.93,130.99
4,34,richards bay,-28.78,78.24,65.0,78.24,17,10.29,32.04
5,53,kapaa,22.08,70.16,73.0,73.40,20,11.41,-159.32
6,92,seminole,28.71,74.26,83.0,75.99,20,11.41,-81.21
7,97,komsomolskiy,40.43,75.72,15.0,77.00,20,4.70,71.72


In [6]:
#cities_lat_lng = requests.get(base_url, params=params)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
#new df with lat, lon , Hotel Name

hotel_df=clean_df[["lon","lat","city name"]].copy()
hotel_df["hotel name"]=""
hotel_df



# # New columng "Coordinates" to join Lat and Long together; easier when pulling from API
hotel_df["Coordinates"] = hotel_df["lat"].map(str) + ", " + hotel_df["lon"].map(str)

# # List to hold the API search results
hotel_result = []
hotel_df
# # Define the search parameters
target_type = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
search_radius = 5000
params = {"keyword" : target_type,
          "radius" : search_radius,
          "key" : g_key}

# Use a for loop to retrieve data and put into hotel_result
try:
    for index, row in hotel_df.iterrows():
        hotel_coord = row["Coordinates"]
        params["location"] = hotel_coord
        nearest_hotel = requests.get(base_url, params = params).json()
        hotel_result.append(nearest_hotel["results"][0]["name"])

except (KeyError, IndexError):
         print("Missing field/result... skipping.")


  






Missing field/result... skipping.


In [22]:
   # print(json.dumps(name_address, indent=4, sort_keys=True))
    hotel_df["Hotel Name"] = hotel_result
    hotel_df


ValueError: Length of values does not match length of index

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
figs


# Display figure
